# 前馈神经网络 (Feedforward Neural Network) - 详细解释与实现

这个Notebook展示了如何从头开始实现一个简单的前馈神经网络，并详细解释了其中涉及的每个步骤、数学原理以及它们在代码中的作用。

我们将以一个经典的XOR问题为例，演示如何训练一个前馈神经网络来学习这种非线性映射。最后，我们还将使用可视化工具绘制神经网络的学习过程以及各层的输出。

## 1. 导入必要的库

我们首先导入 `numpy` 用于矩阵运算，并导入 `matplotlib` 用于数据可视化。

In [3]:
import numpy as np
import matplotlib.pyplot as plt

## 2. 定义激活函数及其导数

在神经网络中，激活函数用于将线性输入映射到非线性输出。我们使用 `sigmoid` 函数作为激活函数，并计算它的导数以便在反向传播时使用。

### Sigmoid函数
Sigmoid函数的数学表达式为：

$$
\sigma(x) = \frac{1}{1 + e^{-x}}
$$

这个函数将任意实数输入映射到 (0, 1) 之间，这对于概率估计非常有用。

### Sigmoid导数
为了在反向传播时更新权重，我们需要计算激活函数的导数。Sigmoid函数的导数表达式为：

$$
\sigma'(x) = \sigma(x) \cdot (1 - \sigma(x))
$$

In [5]:
# 定义激活函数及其导数
def sigmoid(x):
    """Sigmoid 激活函数，将输入映射到 (0, 1) 之间"""
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    """Sigmoid 函数的导数，用于计算反向传播中的梯度"""
    return x * (1 - x)

## 3. 定义前馈神经网络类

我们将创建一个 `NeuralNetwork` 类，其中包含神经网络的初始化、前向传播、反向传播和训练函数。

### 前向传播 (Feedforward)
前向传播的目的是通过网络计算输出。我们通过以下步骤实现：

1. **隐藏层输入计算**：输入层的数据与隐藏层的权重相乘，并加上偏置得到隐藏层的输入。
   $$
   z^{(1)} = X \cdot W^{(1)} + b^{(1)}
   $$

2. **隐藏层输出计算**：通过激活函数处理隐藏层的输入，得到隐藏层的输出。
   $$
   a^{(1)} = \sigma(z^{(1)})
   $$

3. **输出层输入计算**：隐藏层的输出与输出层的权重相乘，并加上偏置得到输出层的输入。
   $$
   z^{(2)} = a^{(1)} \cdot W^{(2)} + b^{(2)}
   $$

4. **输出层输出计算**：通过激活函数处理输出层的输入，得到最终的预测输出。
   $$
   a^{(2)} = \sigma(z^{(2)})
   $$

In [7]:
# 前馈神经网络类
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        # 初始化权重和偏置，随机小值初始化
        self.weights_input_hidden = np.random.randn(input_size, hidden_size)
        self.weights_hidden_output = np.random.randn(hidden_size, output_size)
        self.bias_hidden = np.random.randn(hidden_size)
        self.bias_output = np.random.randn(output_size)
        
        # 保存误差历史记录，便于绘制误差曲线
        self.error_history = []

    def feedforward(self, X):
        """前向传播：从输入层 -> 隐藏层 -> 输出层"""
        # 输入层到隐藏层的线性组合
        self.hidden_input = np.dot(X, self.weights_input_hidden) + self.bias_hidden
        # 隐藏层输出，通过激活函数处理
        self.hidden_output = sigmoid(self.hidden_input)
        
        # 隐藏层到输出层的线性组合
        self.final_input = np.dot(self.hidden_output, self.weights_hidden_output) + self.bias_output
        # 最终输出
        self.final_output = sigmoid(self.final_input)
        
        return self.final_output

### 反向传播 (Backpropagation)
反向传播通过计算输出误差，并通过网络的反向传播来更新权重和偏置。我们通过以下步骤实现：

1. **输出层误差计算**：计算输出层的误差，即期望输出与实际输出的差值。
   $$
   \delta^{(2)} = (y - a^{(2)}) \cdot \sigma'(z^{(2)})
   $$

2. **隐藏层误差计算**：利用输出层的误差反向传播到隐藏层，计算隐藏层的误差。
   $$
   \delta^{(1)} = \delta^{(2)} \cdot W^{(2)T} \cdot \sigma'(z^{(1)})
   $$

3. **更新权重和偏置**：根据误差的梯度更新权重和偏置，使模型的预测更接近期望输出。
   $$
   W^{(2)} \leftarrow W^{(2)} + \eta \cdot a^{(1)T} \cdot \delta^{(2)}
   $$
   $$
   W^{(1)} \leftarrow W^{(1)} + \eta \cdot X^T \cdot \delta^{(1)}
   $$

In [9]:
    def backpropagation(self, X, y, learning_rate):
        """反向传播：根据误差调整权重和偏置"""
        # 前向传播获取输出
        output = self.feedforward(X)
        
        # 计算输出层误差
        error = y - output
        self.error_history.append(np.mean(np.abs(error)))  # 记录误差
        
        # 计算输出层到隐藏层的梯度
        d_output = error * sigmoid_derivative(output)
        
        # 计算隐藏层误差
        error_hidden = d_output.dot(self.weights_hidden_output.T)
        d_hidden = error_hidden * sigmoid_derivative(self.hidden_output)
        
        # 更新权重和偏置
        self.weights_hidden_output += self.hidden_output.T.dot(d_output) * learning_rate
        self.weights_input_hidden += X.T.dot(d_hidden) * learning_rate
        self.bias_output += np.sum(d_output, axis=0) * learning_rate
        self.bias_hidden += np.sum(d_hidden, axis=0) * learning_rate

### 训练 (Training)
训练神经网络的过程是通过多次迭代前向传播和反向传播，不断优化网络的权重和偏置，以降低输出误差。

我们可以通过 `train` 函数指定训练轮数（epochs）和学习率（learning rate），在训练过程中，误差会逐渐减少。

In [20]:
    def train(self, X, y, epochs, learning_rate):
        """训练神经网络：通过多次迭代优化权重和偏置"""
        for epoch in range(epochs):
            self.backpropagation(X, y, learning_rate)
            # 可视化训练过程中的误差
            if epoch % 1000 == 0:
                print(f'Epoch {epoch}, Error: {self.error_history[-1]}')